In [ ]:
# COLAB SETUP
import os

# 1. Clone Code
if not os.path.exists('NeuroMagos'):
    !git clone https://github.com/ChandraguptSharma07/NeuroMagos.git

# 2. Get Data (The Reliable Way - gdown)
# 1. Upload 'Synapse_Dataset.zip' to Google Drive.
# 2. Right Click -> Share -> 'Anyone with the link'.
# 3. Copy Link. It looks like: .../d/1A2B3C.../view
# 4. Paste the ID (the 1A2B3C part) variable below.
file_id = 'PASTE_YOUR_FILE_ID_HERE'

if not os.path.exists('NeuroMagos/Synapse_Dataset'):
    if file_id == 'PASTE_YOUR_FILE_ID_HERE':
        print("Error: You need to paste your Google Drive File ID in the code cell above!")
    else:
        # Install gdown if needed
        !pip install -q gdown
        !gdown --id {file_id} -O Synapse_Dataset.zip
        !unzip -q Synapse_Dataset.zip -d NeuroMagos/

# 3. Enter Directory
%cd NeuroMagos

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

from dataset import SynapseDataset
from model import HybridNeuroMagos

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
# config
bs = 128
lr = 1e-3
epochs = 30

dataset = SynapseDataset('Synapse_Dataset')

# split 80/20
train_len = int(0.8 * len(dataset))
val_len = len(dataset) - train_len
train_ds, val_ds = random_split(dataset, [train_len, val_len])

# num_workers=2 for colab
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=2)
val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False)

len(train_ds), len(val_ds)

In [ ]:
# Improved Config
model = HybridNeuroMagos().to(device)
# Added weight_decay (L2 regularization) to prevent overfitting
opt = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
crit = nn.CrossEntropyLoss()
# Added LR Scheduler: Drops LR by half if Acc doesn't improve for 3 epochs
scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='max', factor=0.5, patience=3)

In [ ]:
history = []
best_acc = 0

for epoch in range(epochs):
    model.train()
    losses = []
    
    # train
    pbar = tqdm(train_dl)
    for x, y in pbar:
        x, y = x.to(device), y.to(device)
        
        opt.zero_grad()
        out = model(x)
        loss = crit(out, y)
        loss.backward()
        opt.step()
        
        losses.append(loss.item())
        pbar.set_description(f"E {epoch+1} | Loss: {np.mean(losses):.4f}")
        
    # val
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in val_dl:
            x, y = x.to(device), y.to(device)
            out = model(x)
            pred = out.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
            
    acc = correct / total
    print(f"Val Acc: {acc:.4f}")
    
    # Step the scheduler based on Val Acc
    scheduler.step(acc)
    
    if acc > best_acc:
        torch.save(model.state_dict(), 'best_model.pth')
        best_acc = acc
        print("Saved.")
        
    history.append(acc)

In [ ]:
plt.plot(history)
plt.title('Val Acc')
plt.show()